In [7]:
import torch
import random
import numpy as np
from tqdm import tqdm
from datasets import Dataset
from transformers import (
    BertTokenizer,
    DataCollatorForWholeWordMask,
)

In [8]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

In [9]:
seed = 3407
seed_everything(seed)
model_path = 'bert-base-chinese'
vocab_path = model_path
file_path = "./longStn.txt"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
max_length = 512
mlm_probability = 0.15
wwm=True

In [10]:
train_data = {"sentence": []}
with open(file_path, 'r', encoding="utf-8") as f:
    lines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]
    for line in tqdm(lines):
        train_data["sentence"].append(line)

100%|██████████| 49/49 [00:00<?, ?it/s]


In [11]:
dataset = Dataset.from_dict(train_data)
dataset

Dataset({
    features: ['sentence'],
    num_rows: 49
})

In [12]:
tokenizer = BertTokenizer.from_pretrained(vocab_path)

'HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-chinese/resolve/main/vocab.txt (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001CE8C99BD60>, 'Connection to huggingface.co timed out. (connect timeout=10)'))' thrown while requesting HEAD https://huggingface.co/bert-base-chinese/resolve/main/vocab.txt


In [13]:
dataset = dataset.map(lambda example:{"input_ids":tokenizer(example["sentence"], truncation=True, max_length=max_length).input_ids},batched=True)
dataset

Parameter 'function'=<function <lambda> at 0x000001CE8CA6BA60> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
  0%|          | 0/1 [00:00<?, ?ba/s]


Dataset({
    features: ['sentence', 'input_ids'],
    num_rows: 49
})

In [14]:
dataset.save_to_disk("./dataset_temp_long")

In [15]:
from datasets import load_from_disk
dataset = load_from_disk("./dataset_temp_long")
dataset

Dataset({
    features: ['sentence', 'input_ids'],
    num_rows: 49
})

In [16]:
from get_chinese_ref import prepare_ref
from ltp import LTP

if wwm:
    ltp = LTP().to(device)#这一句要加载180MB的模型并初始化，所以很慢，要大概五秒钟时间
    dataset = dataset.map(lambda example:{"chinese_ref":prepare_ref(example["sentence"], ltp, tokenizer)},batched=True)
dataset

Loading weights from local directory


  0%|          | 0/1 [00:05<?, ?ba/s]


Dataset({
    features: ['sentence', 'input_ids', 'chinese_ref'],
    num_rows: 49
})

In [17]:
dataset["sentence"][0]

'谷物联合收获机自动测产系统设计-基于变权分层激活扩散模型[PAD]联合收割机[PAD]测产系统[PAD]变权分层[PAD]激活扩散[PAD]为了使联合收割机具有自动测产功能，提出了一种基于变权分层激活扩散的产量预测误差剔除模型，并使用单片机设计了联合收获机测产系统。测产系统的主要功能是：在田间进行作业时，收割机可以测出当前的运行速度、收获面积及谷物的总体产量。数据的采集使用霍尔传感器和电容压力传感器，具有较高的精度。模拟信号的处理选用了 ADC0804差分式 A／D转换芯片，可以有效地克服系统误差，数据传送到单片机处理中心，对每一次转换都进行一次判断，利用变权分层激活扩散模型剔除误差较大的数据，通过计算将数据最终在LCD显示屏进行显示。将系统应用在了收割机上，通过测试得到了谷物产量的测量值，并与真实值进行比较，验证了系统的可靠性。'

In [18]:
tokenizer.decode(dataset["input_ids"][0])

'[CLS] 谷 物 联 合 收 获 机 自 动 测 产 系 统 设 计 - 基 于 变 权 分 层 激 活 扩 散 模 型 [PAD] 联 合 收 割 机 [PAD] 测 产 系 统 [PAD] 变 权 分 层 [PAD] 激 活 扩 散 [PAD] 为 了 使 联 合 收 割 机 具 有 自 动 测 产 功 能 ， 提 出 了 一 种 基 于 变 权 分 层 激 活 扩 散 的 产 量 预 测 误 差 剔 除 模 型 ， 并 使 用 单 片 机 设 计 了 联 合 收 获 机 测 产 系 统 。 测 产 系 统 的 主 要 功 能 是 ： 在 田 间 进 行 作 业 时 ， 收 割 机 可 以 测 出 当 前 的 运 行 速 度 、 收 获 面 积 及 谷 物 的 总 体 产 量 。 数 据 的 采 集 使 用 霍 尔 传 感 器 和 电 容 压 力 传 感 器 ， 具 有 较 高 的 精 度 。 模 拟 信 号 的 处 理 选 用 了 [UNK] 差 分 式 [UNK] ／ [UNK] 转 换 芯 片 ， 可 以 有 效 地 克 服 系 统 误 差 ， 数 据 传 送 到 单 片 机 处 理 中 心 ， 对 每 一 次 转 换 都 进 行 一 次 判 断 ， 利 用 变 权 分 层 激 活 扩 散 模 型 剔 除 误 差 较 大 的 数 据 ， 通 过 计 算 将 数 据 最 终 在 [UNK] 显 示 屏 进 行 显 示 。 将 系 统 应 用 在 了 收 割 机 上 ， 通 过 测 试 得 到 了 谷 物 产 量 的 测 量 值 ， 并 与 真 实 值 进 行 比 较 ， 验 证 了 系 统 的 可 靠 性 。 [SEP]'

In [19]:
dataset["chinese_ref"][0]

[2,
 4,
 6,
 7,
 9,
 11,
 13,
 15,
 18,
 20,
 22,
 24,
 25,
 26,
 28,
 31,
 33,
 34,
 37,
 39,
 42,
 44,
 47,
 48,
 49,
 52,
 55,
 57,
 58,
 60,
 62,
 64,
 66,
 69,
 74,
 76,
 78,
 80,
 81,
 82,
 85,
 87,
 89,
 91,
 93,
 97,
 99,
 100,
 102,
 105,
 107,
 108,
 110,
 112,
 115,
 117,
 120,
 122,
 127,
 129,
 131,
 135,
 136,
 138,
 140,
 142,
 145,
 147,
 150,
 152,
 155,
 158,
 160,
 163,
 166,
 168,
 170,
 172,
 173,
 176,
 178,
 180,
 181,
 184,
 189,
 192,
 194,
 197,
 199,
 209,
 211,
 214,
 216,
 219,
 221,
 223,
 226,
 228,
 231,
 232,
 234,
 236,
 243,
 246,
 250,
 253,
 255,
 257,
 259,
 260,
 261,
 263,
 265,
 267,
 272,
 275,
 277,
 280,
 282,
 286,
 287,
 289,
 291,
 295,
 297,
 301,
 302,
 306,
 308,
 310,
 313,
 314,
 315,
 318,
 319,
 323,
 325,
 327,
 329,
 332,
 335,
 338,
 339]

In [20]:
data_collator = DataCollatorForWholeWordMask(tokenizer=tokenizer, mlm=True, mlm_probability=mlm_probability)

In [21]:
tokenizer.decode(data_collator([dataset[0]])["labels"][0])

'[UNK] [UNK] [UNK] [UNK] [UNK] 收 获 机 [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] 激 活 扩 散 模 型 [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] 测 产 [UNK] [UNK] [UNK] 变 权 [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] 使 [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] 了 [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] 了 [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] 收 割 机 [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] 。 [UNK] [UNK] 的 [UNK] [UNK] [UNK] [UNK] 霍 尔 [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] 压 力 传 感 器 [UNK] [UNK] [UNK] [

In [22]:
dataset = dataset.remove_columns("sentence")

In [23]:
dataset.save_to_disk("./dataset_long")
